In [ ]:
#### dokumentasi yang sudah disiapkan DataTalksClub
#### https://github.com/DataTalksClub/data-engineering-zoomcamp/tree/main/01-docker-terraform/2_docker_sql

#### download data dari link berikut:
wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2021-01.csv.gz

#### letakkan data di folder berikut:
D:\data_integration\dezoomcamp2024\01-docker-terraform\2_docker_sql\ny_taxi_postgres_data

#### jalankan docker menggunakan perintah berikut:
docker run -it \
  --name zoomcamp_postgres \
  -e POSTGRES_USER="root" \
  -e POSTGRES_PASSWORD="root" \
  -e POSTGRES_DB="ny_taxi" \
  -v D:/data_integration/dezoomcamp2024/01-docker-terraform/2_docker_sql/ny_taxi_postgres_data:/var/lib/postgresql/ny_taxi \
  -p 5433:5432 \
  postgres:13

############################################################################################################################################

CLI for Postgres
Installing pgcli
  
  pip install pgcli

############################################################################################################################################

Using pgcli to connect to Postgres

pgcli -h localhost -p 5433 -u root -d ny_taxi

############################################################################################################################################

Running pgAdmin

docker run -it \
  --name zoomcamp_pgadmin \
  -e PGADMIN_DEFAULT_EMAIL="admin@admin.com" \
  -e PGADMIN_DEFAULT_PASSWORD="root" \
  -p 8181:80 \
  dpage/pgadmin4

############################################################################################################################################

Running Postgres and pgAdmin together
Create a network

docker network create pg-network

############################################################################################################################################

docker run -it \
  --name zoomcamp_postgres \
  -e POSTGRES_USER="root" \
  -e POSTGRES_PASSWORD="root" \
  -e POSTGRES_DB="ny_taxi" \
  -v D:/data_integration/dezoomcamp2024/01-docker-terraform/2_docker_sql/ny_taxi_postgres_data:/var/lib/postgresql/ny_taxi \
  -p 5433:5432 \
  --network=pg-network \
  --name pg-database \
  postgres:13

############################################################################################################################################

Running pgAdmin

docker run -it \
  --name zoomcamp_pgadmin \
  -e PGADMIN_DEFAULT_EMAIL="admin@admin.com" \
  -e PGADMIN_DEFAULT_PASSWORD="root" \
  -p 8181:80 \
  --network=pg-network \
  --name pgadmin-2 \
  dpage/pgadmin4

############################################################################################################################################

Data ingestion
Running locally

URL="https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2021-01.csv.gz"

python ingest_data.py \
  --user=root \
  --password=root \
  --host=localhost \
  --port=5433 \
  --db=ny_taxi \
  --table_name=yellow_taxi_trips \
  --url=${URL}

############################################################################################################################################

Build the image

docker build -t taxi_ingest:v001 .

############################################################################################################################################

Run the script with Docker

URL="https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2021-01.csv.gz"

docker run -it \
  --network=pg-network \
  taxi_ingest:v001 \
    --user=root \
    --password=root \
    --host=pg-database \
    --port=5433 \
    --db=ny_taxi \
    --table_name=yellow_taxi_trips \
    --url=${URL}

############################################################################################################################################

Docker-Compose
Run it:

docker-compose up
Run in detached mode:

docker-compose up -d
Shutting it down:

docker-compose down

############################################################################################################################################

Note: to make pgAdmin configuration persistent, create a folder data_pgadmin. Change its permission via

sudo chown 5050:5050 data_pgadmin
and mount it to the /var/lib/pgadmin folder:

services:
  pgadmin:
    image: dpage/pgadmin4
    volumes:
      - ./data_pgadmin:/var/lib/pgadmin
    ...

############################################################################################################################################

services:
  postgres:
    image: postgres:13
    environment:
      POSTGRES_USER: airflow
      POSTGRES_PASSWORD: airflow
      POSTGRES_DB: airflow
    volumes:
      - postgres-db-volume:/var/lib/postgresql/data
    healthcheck:
      test: ["CMD", "pg_isready", "-U", "airflow"]
      interval: 5s
      retries: 5
    restart: always

docker run -it \
  -e POSTGRES_USER="root" \
  -e POSTGRES_PASSWORD="root" \
  -e POSTGRES_DB="ny_taxi" \
  -v d:/data_integration/dezoomcamp2024/01-docker-terraform/2_docker_sql \
  -p 5432:5432 \
  postgres:13

---------step dari sini----------
docker run -it \
  -e POSTGRES_USER="root" \
  -e POSTGRES_PASSWORD="root" \
  -e POSTGRES_DB="ny_taxi" \
  -v $(pwd)/ny_taxi_postgres_data:/var/lib/postgresql/data \
  -p 5432:5432 \
  postgres:13

pgcli -h localhost -p 5432 -u root -d ny_taxi

